In [1]:
# Notebook parameters. Values here are for development only and 
# will be overridden when running via snakemake and papermill.
country = 'ML'
use_gcs_cache = False
dask_scheduler = "threads"

In [2]:
# Parameters
country = "CF"


In [3]:
from pyprojroot import here
import geopandas as gpd
import dask
dask.config.set(scheduler=dask_scheduler);

# Central African Republic

The map below shows regions where mosquito genomic data are available and genome-wide 
selection scans have been performed.

Click on a region to see more information about mosquito cohorts for which data are 
available.

In [4]:
gdf_cohorts = gpd.read_file(here() / "build" / "final_cohorts.geojson").set_index('cohort_id')
gdf_cohorts = gdf_cohorts.query(f"country_alpha2 == '{country}'")

In [5]:
from ipyleaflet import Map, Marker, basemaps, AwesomeIcon
from ipywidgets import HTML

center = gdf_cohorts[['latitude', 'longitude']].mean().to_list()
m = Map(center=center, zoom=7, basemap=basemaps.OpenTopoMap)

for shapeID in gdf_cohorts.shapeID.unique():

    df = gdf_cohorts.query("shapeID == @shapeID")    
    html_text = '<br>'.join([
                                f'<a href="../cohort/{coh_id}.html">{row.cohort_label} (n={row.cohort_size})</a>' 
                                for coh_id, row in df.iterrows()
                            ])

    lat, long = df[['representative_lat', 'representative_lon']].drop_duplicates().values[0]

    icon1 = AwesomeIcon(
        name='bug',
        marker_color="gray",
        icon_color='white',
        spin=False
    )
    
    marker = Marker(location=(lat, long), icon=icon1, draggable=False, opacity=0.7, color="gray")
    m.add_layer(marker);
    
    message2 = HTML()
    message2.value = html_text
    marker.popup = message2

m

Map(center=[4.367, 18.583], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

<br></br>

In [6]:
def make_clickable(url, name):
    return '<a href="{}" rel="noopener noreferrer" target="_blank">{}</a>'.format(url,name)

In [7]:
gdf_cohorts = gdf_cohorts.reset_index()
gdf_cohorts['url'] = "../cohort/" + gdf_cohorts['cohort_id'] + ".html"
gdf_cohorts['cohort_label'] = gdf_cohorts.apply(lambda x: make_clickable(x['url'], x['cohort_label']), axis=1)

gdf_cohorts = gdf_cohorts[['cohort_label', 'cohort_size', 'admin1_name', 'admin2_name', 'taxon', 'year', 'quarter']]
gdf_cohorts = gdf_cohorts.rename(columns={'cohort_label':'Cohort label'}).set_index('Cohort label')
gdf_cohorts.columns = ['Cohort size', 'Admin1 area', 'Admin2 area', 'Taxon', 'Year', 'Quarter']
gdf_cohorts.style

,Cohort size,Admin1 area,Admin2 area,Taxon,Year,Quarter
Cohort label,,,,,,
Central African Republic / Bangui / gambiae / 1994 / Q1,53,Bangui,Bangui,gambiae,1994,1
